# Data processing/feature engineering

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [137]:
#read clean dataset
data = pd.read_csv('data/dataset_clean.csv')

In [138]:
data

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97104,443847548,100162476,AfricanAmerican,Male,[70-80),1,3,7,3,51,...,No,Down,No,No,No,No,No,Ch,Yes,>30
97105,443847782,74694222,AfricanAmerican,Female,[80-90),1,4,5,5,33,...,No,Steady,No,No,No,No,No,No,Yes,NO
97106,443854148,41088789,Caucasian,Male,[70-80),1,1,7,1,53,...,No,Down,No,No,No,No,No,Ch,Yes,NO
97107,443857166,31693671,Caucasian,Female,[80-90),2,3,7,10,45,...,No,Up,No,No,No,No,No,Ch,Yes,NO


#### Removing drug features

There are multiple drug features in the dataset that were never prescribed to a patient. We will remove all instances of such features if more than 95% of the patients were not presecribed.

In [139]:
#list of all drug features
drugs = np.array(['metformin',
        'repaglinide',
        'nateglinide',
        'chlorpropamide',
        'glimepiride',
        'acetohexamide',
        'glipizide',
        'glyburide',
        'tolbutamide',
        'pioglitazone',
        'rosiglitazone',
        'acarbose',
        'miglitol',
        'troglitazone',
        'tolazamide',
        'examide',
        'citoglipton',
        'insulin',
        'glyburide-metformin',
        'glipizide-metformin',
        'glimepiride-pioglitazone',
        'metformin-rosiglitazone',
        'metformin-pioglitazone'])

In [140]:
#dropping drug columns if more than 95% of the values are No = not prescribed
dropped_drug_cols = []

for d in drugs:
    counts = data[d].value_counts()
    if (counts['No']>0.95*len(data)):
        dropped_drug_cols.append(d)

#storing a list of features that remain
accepted_drug_cols = np.setdiff1d(drugs, dropped_drug_cols).tolist()

data = data.drop(columns = dropped_drug_cols)


#### Grouping the industry standard codes

In [141]:
data['admission_urg_ind'] = (data['admission_type_id'].isin([1,2,7])).astype(int)
data['admission_nurg_ind'] = (data['admission_type_id'].isin([3,4])).astype(int)
data['admission_none_ind'] = (data['admission_type_id'].isin([5,6,8])).astype(int)

data= data.drop(columns='admission_type_id')

In [142]:
data['discharge_shortterm_ind'] = (data['discharge_disposition_id'].isin([1,2,9,10,16,17,28,29])).astype(int)
data['discharge_longterm_ind'] = (data['discharge_disposition_id'].isin([3,4,5,6,7,8,12,15,22,23,24,30,28])).astype(int)
data['discharge_none_ind'] = (data['discharge_disposition_id'].isin([18,25,26])).astype(int)

data = data.drop(columns='discharge_disposition_id')

In [143]:
data['admission_source_1'] = (data['admission_source_id'].isin([1,2,3,4,5,6,10,18,19,22,25,26])).astype(int)
data['admission_source_2'] = (data['admission_source_id'].isin([8,11,12,13,14,23,24])).astype(int)
data['admission_source_3'] = (data['admission_source_id'].isin([9,15,17,20,21])).astype(int)

data = data.drop(columns='admission_source_id')


#### Encoding the feature: `race`

In [144]:
data['race'].value_counts()

race
Caucasian          74220
AfricanAmerican    18772
Hispanic            2017
Other               1472
Asian                628
Name: count, dtype: int64

In [145]:
#One-hot encoding manually. Not using get_dummies to control the column name

data['caucasian_ind'] = data['race'].isin(['Caucasian']).astype(int)
data['africanamerican_ind'] = data['race'].isin(['AfricanAmerican']).astype(int)
data['hispanic_ind'] = data['race'].isin(['Hispanic']).astype(int)
data['asian_ind'] = data['race'].isin(['Asian']).astype(int)
data['otherrace_ind'] = data['race'].isin(['Other']).astype(int)

dropped_columns = ['patient_nbr', 'encounter_id']

#### Encoding the feature: `gender`

In [146]:
data['gender'].value_counts()

gender
Female             52336
Male               44772
Unknown/Invalid        1
Name: count, dtype: int64

In [147]:
#dropping entry with Unkown/invalid gender
data = data.drop(data['gender'].loc[data['gender'] == 'Unknown/Invalid'].index)

data['female_ind'] = data['gender'].isin(['Female']).astype(int)
data['male_ind'] = data['gender'].isin(['Male']).astype(int)

dropped_columns.append('gender')

#### Encoding the feature: `age`

In [148]:
data.age.value_counts()

age
[70-80)     24743
[60-70)     21572
[50-60)     16701
[80-90)     16049
[40-50)      9389
[30-40)      3688
[90-100)     2522
[20-30)      1603
[10-20)       682
[0-10)        159
Name: count, dtype: int64

In [149]:
#Using Tam's code here
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
#it's cool to use LabelEncoder here because our 'age' column actually sorted in ascending 'alphabetical' order
#faircare_pd = pd.DataFrame(faircare)
#this is a choice to use LabelEncoder instead of replacing with a dictionary. We'll use a dictionary for replacing other column entries with numbers
data['age'] = label_encoder.fit_transform(data['age']).astype(int)


#### Encoding the diagnoses

Features `diag_1`, `diag_2`, and `diag_3` contain >800 distinct alphanumeric diagnosis codes (ICD-9). We will group them as follows:
- 001–139: infectious and parasitic diseases
- 140–239: neoplasms
- 240–279: endocrine, nutritional and metabolic diseases, and immunity disorders
- 280–289: diseases of the blood and blood-forming organs
- 290–319: mental disorders
- 320–389: diseases of the nervous system and sense organs
- 390–459: diseases of the circulatory system
- 460–519: diseases of the respiratory system
- 520–579: diseases of the digestive system
- 580–629: diseases of the genitourinary system
- 630–679: complications of pregnancy, childbirth, and the puerperium
- 680–709: diseases of the skin and subcutaneous tissue
- 710–739: diseases of the musculoskeletal system and connective tissue
- 740–759: congenital anomalies
- 760–779: certain conditions originating in the perinatal period
- 780–799: symptoms, signs, and ill-defined conditions
- 800–999: injury and poisoning
- E and V codes: external causes of injury and supplemental classification

Note that there are multiple distinct codes for external causes (E000 - E999) as well as for supplementary factors (V01 - V82), however they will be grouped together.

Additionally, we will add features corresponding to diagnoses which are correlated with most readmissions.

In [150]:
#Finding the diagnoses corresponding to the highest number of readmits
readmit_diags = data.loc[(data['readmitted']=='<30')][['diag_1','diag_2','diag_3']]
readmit_diags.melt(value_name='diag')['diag'].value_counts().head(20)

diag
428       2329
276       1641
250       1409
427       1249
414       1131
401        971
403        882
599        760
496        709
585        607
491        543
250.6      542
486        519
707        513
584        510
780        478
682        473
410        449
250.02     447
786        393
Name: count, dtype: int64

In [151]:
# Note that diag codes 250.xx are diabetic conditions, so we won't consider them
# Only considering diagnoses with >500 instances of readmission
data['readmit_diag_1'] = ((data['diag_1']=='428')|
                          (data['diag_2']=='428')|
                          (data['diag_3']=='428')).astype(int)

data['readmit_diag_2'] = ((data['diag_1']=='276')|
                          (data['diag_2']=='276')|
                          (data['diag_3']=='276')).astype(int)

data['readmit_diag_3'] = ((data['diag_1']=='427')|
                          (data['diag_2']=='427')|
                          (data['diag_3']=='427')).astype(int)

data['readmit_diag_4'] = ((data['diag_1']=='414')|
                          (data['diag_2']=='414')|
                          (data['diag_3']=='414')).astype(int)

data['readmit_diag_5'] = ((data['diag_1']=='401')|
                          (data['diag_2']=='401')|
                          (data['diag_3']=='401')).astype(int)

data['readmit_diag_6'] = ((data['diag_1']=='403')|
                          (data['diag_2']=='403')|
                          (data['diag_3']=='403')).astype(int)

data['readmit_diag_7'] = ((data['diag_1']=='599')|
                          (data['diag_2']=='599')|
                          (data['diag_3']=='599')).astype(int)

data['readmit_diag_8'] = ((data['diag_1']=='496')|
                          (data['diag_2']=='496')|
                          (data['diag_3']=='496')).astype(int)

data['readmit_diag_9'] = ((data['diag_1']=='585')|
                          (data['diag_2']=='585')|
                          (data['diag_3']=='585')).astype(int)

data['readmit_diag_10'] = ((data['diag_1']=='491')|
                           (data['diag_2']=='491')|
                           (data['diag_3']=='491')).astype(int)

data['readmit_diag_11'] = ((data['diag_1']=='486')|
                           (data['diag_2']=='486')|
                           (data['diag_3']=='486')).astype(int)

data['readmit_diag_12'] = ((data['diag_1']=='707')|
                           (data['diag_2']=='707')|
                           (data['diag_3']=='707')).astype(int)

data['readmit_diag_13'] = ((data['diag_1']=='584')|
                           (data['diag_2']=='584')|
                           (data['diag_3']=='584')).astype(int)

In [152]:
print(len(data))
data['patient_nbr'].nunique()

97108


68166

An interesting observation about the data is that although there are $\sim 10^5$ total data points, there are only about $\sim 7 \times 10^4$ distinct patients whose data we have. Therefore, we will be adding features corresponding to their medical histories specifically.
This idea was inspired by [this project](https://github.com/lelandburrill/diabetes_readmission) (see references).

In [153]:
readmit_diags = ['readmit_diag_1',
                 'readmit_diag_2',
                 'readmit_diag_3',
                 'readmit_diag_4',
                 'readmit_diag_5',
                 'readmit_diag_6',
                 'readmit_diag_7',
                 'readmit_diag_8',
                 'readmit_diag_9',
                 'readmit_diag_10',
                 'readmit_diag_11',
                 'readmit_diag_12',
                 'readmit_diag_13']

In [154]:
#storing the number of hig-risk diagnoses for each patient
data['risk_diag_ct'] = data[readmit_diags].sum(axis=1)
data['pt_risk_diag_ct'] = data.groupby('patient_nbr')['risk_diag_ct'].transform('sum')

data = data.drop(columns='risk_diag_ct')

In [155]:
#storing the total number of unique diagnoses for each patient
diag_info = data.melt(id_vars='patient_nbr', var_name='diag', value_vars =['diag_1','diag_2','diag_3'], value_name='pt_diag_ct')
data = data.merge(diag_info.groupby('patient_nbr')['pt_diag_ct'].nunique().reset_index(), on ='patient_nbr')

del diag_info

In [156]:
#coding alpha-numeric codes first
data['d_external_index'] = ( (data['diag_1'].str[0] == 'E') |
                             (data['diag_2'].str[0] == 'E') |
                             (data['diag_3'].str[0] == 'E')).astype(int)

data['d_supplementary_index'] = ((data['diag_1'].str[0] == 'V') |
                                 (data['diag_2'].str[0] == 'V') |
                                 (data['diag_3'].str[0] == 'V')).astype(int)

#no diagnosis
#data['d_nodiag_ind'] = ( (data['diag_1'] == 'No') |
#                         (data['diag_2'] == 'No') |
#                         (data['diag_3'] == 'No')).astype(int)

In [157]:
data['diag_1_n'] = data['diag_1'].apply(pd.to_numeric, errors='coerce')
data['diag_2_n'] = data['diag_2'].apply(pd.to_numeric, errors='coerce')
data['diag_3_n'] = data['diag_3'].apply(pd.to_numeric, errors='coerce')

data['diag_1_n'].dropna()
data['diag_2_n'].dropna()
data['diag_3_n'].dropna()

dropped_columns.extend(['diag_1','diag_2','diag_3'])

In [158]:
#infectious and parasitic diseases
data['d_infect_ind'] = ( ((data['diag_1_n'] >= 1) & (data['diag_1_n'] <= 139)) |
                         ((data['diag_2_n'] >= 1) & (data['diag_2_n'] <= 139)) |
                         ((data['diag_3_n'] >= 1) & (data['diag_3_n'] <= 139)) ).astype(int)

#neoplasms
data['d_neoplasm_ind'] = ( ((data['diag_1_n'] >= 140) & (data['diag_1_n'] <= 239)) |
                           ((data['diag_2_n'] >= 140) & (data['diag_2_n'] <= 239)) |
                           ((data['diag_3_n'] >= 140) & (data['diag_3_n'] <= 239)) ).astype(int)

#endocrine, nutritional and metabolic diseases, and immunity disorders
data['d_endocrine_ind'] = ( ((data['diag_1_n'] >= 240) & (data['diag_1_n'] <= 279)) |
                            ((data['diag_2_n'] >= 240) & (data['diag_2_n'] <= 279)) |
                            ((data['diag_3_n'] >= 240) & (data['diag_3_n'] <= 279)) ).astype(int)

#diseases of the blood and blood-forming organs
data['d_blood_ind'] = ( ((data['diag_1_n'] >= 280) & (data['diag_1_n'] <= 289)) |
                        ((data['diag_2_n'] >= 280) & (data['diag_2_n'] <= 289)) |
                        ((data['diag_3_n'] >= 280) & (data['diag_3_n'] <= 289)) ).astype(int)

#mental disorders
data['d_mental_ind'] = ( ((data['diag_1_n'] >= 290) & (data['diag_1_n'] <= 319)) |
                         ((data['diag_2_n'] >= 290) & (data['diag_2_n'] <= 319)) |
                         ((data['diag_3_n'] >= 290) & (data['diag_3_n'] <= 319)) ).astype(int)

#diseases of the nervous system and sense organs
data['d_nervous_ind'] = ( ((data['diag_1_n'] >= 320) & (data['diag_1_n'] <= 389)) |
                          ((data['diag_2_n'] >= 320) & (data['diag_2_n'] <= 389)) |
                          ((data['diag_3_n'] >= 320) & (data['diag_3_n'] <= 389)) ).astype(int)

#diseases of the circulatory system
data['d_circulatory_ind'] = ( ((data['diag_1_n'] >= 390) & (data['diag_1_n'] <= 459)) |
                              ((data['diag_2_n'] >= 390) & (data['diag_2_n'] <= 459)) |
                              ((data['diag_3_n'] >= 390) & (data['diag_3_n'] <= 459)) ).astype(int)

#diseases of the respiratory system
data['d_respiratory_ind'] = ( ((data['diag_1_n'] >= 460) & (data['diag_1_n'] <= 519)) |
                              ((data['diag_2_n'] >= 460) & (data['diag_2_n'] <= 519)) |
                              ((data['diag_3_n'] >= 460) & (data['diag_3_n'] <= 519)) ).astype(int)

#diseases of the digestive system
data['d_digestive_ind'] = ( ((data['diag_1_n'] >= 520) & (data['diag_1_n'] <= 579)) |
                            ((data['diag_2_n'] >= 520) & (data['diag_2_n'] <= 579)) |
                            ((data['diag_3_n'] >= 520) & (data['diag_3_n'] <= 579)) ).astype(int)

#diseases of the genitourinary system
data['d_genitourinary_ind'] = ( ((data['diag_1_n'] >= 580) & (data['diag_1_n'] <= 629)) |
                                ((data['diag_2_n'] >= 580) & (data['diag_2_n'] <= 629)) |
                                ((data['diag_3_n'] >= 580) & (data['diag_3_n'] <= 629)) ).astype(int)

#complications of pregnancy, childbirth, and the puerperium
data['d_pegnancy_ind'] = ( ((data['diag_1_n'] >= 630) & (data['diag_1_n'] <= 679)) |
                           ((data['diag_2_n'] >= 630) & (data['diag_2_n'] <= 679)) |
                           ((data['diag_3_n'] >= 630) & (data['diag_3_n'] <= 679)) ).astype(int)

#diseases of the skin and subcutaneous tissue
data['d_skin_ind'] = ( ((data['diag_1_n'] >= 680) & (data['diag_1_n'] <= 709)) |
                       ((data['diag_2_n'] >= 680) & (data['diag_2_n'] <= 709)) |
                       ((data['diag_3_n'] >= 680) & (data['diag_3_n'] <= 709)) ).astype(int)

#diseases of the musculoskeletal system and connective tissue
data['d_musculo_ind'] = ( ((data['diag_1_n'] >= 710) & (data['diag_1_n'] <= 739)) |
                          ((data['diag_2_n'] >= 710) & (data['diag_2_n'] <= 739)) |
                          ((data['diag_3_n'] >= 710) & (data['diag_3_n'] <= 739)) ).astype(int)

#congenital anomalies
data['d_congenital_ind'] = ( ((data['diag_1_n'] >= 740) & (data['diag_1_n'] <= 759)) |
                             ((data['diag_2_n'] >= 740) & (data['diag_2_n'] <= 759)) |
                             ((data['diag_3_n'] >= 740) & (data['diag_3_n'] <= 759)) ).astype(int)

#certain conditions originating in the perinatal period
data['d_perinatal_ind'] = ( ((data['diag_1_n'] >= 760) & (data['diag_1_n'] <= 779)) |
                            ((data['diag_2_n'] >= 760) & (data['diag_2_n'] <= 779)) |
                            ((data['diag_3_n'] >= 760) & (data['diag_3_n'] <= 779)) ).astype(int)

#symptoms, signs, and ill-defined conditions
data['d_symptoms_ind'] = ( ((data['diag_1_n'] >= 780) & (data['diag_1_n'] <= 799)) |
                           ((data['diag_2_n'] >= 780) & (data['diag_2_n'] <= 799)) |
                           ((data['diag_3_n'] >= 780) & (data['diag_3_n'] <= 799)) ).astype(int)

#injury and poisoning
data['d_injury_ind'] = ( ((data['diag_1_n'] >= 800) & (data['diag_1_n'] <= 999)) |
                         ((data['diag_2_n'] >= 800) & (data['diag_2_n'] <= 999)) |
                         ((data['diag_3_n'] >= 800) & (data['diag_3_n'] <= 999)) ).astype(int)


dropped_columns.extend(['diag_1_n','diag_2_n','diag_3_n'])

#### Encoding test results

In [159]:
data['A1Cresult'].value_counts()

A1Cresult
No      80694
>8       7884
Norm     4837
>7       3693
Name: count, dtype: int64

In [160]:
data['max_glu_serum'].value_counts()

max_glu_serum
No      92010
Norm     2520
>200     1400
>300     1178
Name: count, dtype: int64

In [161]:
A1c_map = {">8" : 3,
           ">7" : 2,
           "Norm" : 1,
           "No" : 0}

max_glu_serum_map = {">300" : 3,
                     ">200" : 2,
                     "Norm" : 1,
                     "No" : 0}

In [162]:
#mapping the test results to appropriate integers
data['A1Cresult'] = data['A1Cresult'].map(A1c_map)
data['max_glu_serum'] = data['max_glu_serum'].map(max_glu_serum_map)

In [163]:
data.shape

(97108, 81)

#### Encoding drug features

In [164]:
accepted_drug_cols

['glimepiride',
 'glipizide',
 'glyburide',
 'insulin',
 'metformin',
 'pioglitazone',
 'rosiglitazone']

In [165]:
drugs_map = {
    'No': 0,
    'Down': 1,
    'Steady': 2,
    'Up': 3
}
for d in accepted_drug_cols:
    data[d] = data[d].map(drugs_map)

#### Encoding diabetes med features

In [166]:
data['change'].value_counts()

change
No    52071
Ch    45037
Name: count, dtype: int64

In [167]:
data['diabetesMed'].value_counts()

diabetesMed
Yes    74873
No     22235
Name: count, dtype: int64

In [168]:
data['dia_med_change_ind'] = ( data['change'] == "Ch" ).astype(int)
data['dia_med_nochange_ind'] = ( data['change'] == "No" ).astype(int)
data['diabetesMed'] = ( data['diabetesMed'] == "Yes" ).astype(int)
dropped_columns.extend(['change'])

#### Unique patient histories

In [169]:
#Counting each admission type
data['pt_admission_urg_ct'] = data.groupby('patient_nbr')['admission_urg_ind'].transform('sum')
data['admission_nurg_ct'] = data.groupby('patient_nbr')['admission_nurg_ind'].transform('sum')
data['admission_none_ct'] = data.groupby('patient_nbr')['admission_none_ind'].transform('sum')

#Counting types of discharge
data['pt_discharge_shortterm_ct'] = data.groupby('patient_nbr')['discharge_shortterm_ind'].transform('sum')
data['pt_discharge_longterm_ct'] = data.groupby('patient_nbr')['discharge_longterm_ind'].transform('sum')
data['pt_discharge_none_ct'] = data.groupby('patient_nbr')['discharge_none_ind'].transform('sum')

#Counting different sources of admission
data['pt_admission_source_1_ct'] = data.groupby('patient_nbr')['admission_source_1'].transform('sum')
data['pt_admission_source_2_ct'] = data.groupby('patient_nbr')['admission_source_2'].transform('sum')
data['pt_admission_source_3_ct'] = data.groupby('patient_nbr')['admission_source_3'].transform('sum')

In [170]:
#storing the totals of all numerical features for each patient
data['pt_time_tot'] = data.groupby('patient_nbr')['time_in_hospital'].transform('sum')
data['pt_lab_procedure_tot'] = data.groupby('patient_nbr')['num_lab_procedures'].transform('sum')
data['pt_procedure_tot'] = data.groupby('patient_nbr')['num_procedures'].transform('sum')
data['pt_med_tot'] = data.groupby('patient_nbr')['num_medications'].transform('sum')
data['pt_outp_tot'] = data.groupby('patient_nbr')['number_outpatient'].transform('sum')
data['pt_inp_tot'] = data.groupby('patient_nbr')['number_inpatient'].transform('sum')
data['pt_ER_tot'] = data.groupby('patient_nbr')['number_emergency'].transform('sum')
data['pt_diag_tot'] = data.groupby('patient_nbr')['number_diagnoses'].transform('sum')

In [171]:
#counting past re-admissions
data['readmit_lt30'] = (data['readmitted'].isin(['<30']).astype(int))
data['readmit_gt30'] = (data['readmitted'].isin(['>30']).astype(int))
data['readmit_none'] = (data['readmitted'].isin(['NO']).astype(int))

data['past_readmits_lt30'] = data.groupby('patient_nbr')['readmit_lt30'].transform('sum')
#data.loc[data['past_readmits_lt30'] > 0, 'past_readmits_lt30'] -= 1
data['past_readmits_gt30'] = data.groupby('patient_nbr')['readmit_gt30'].transform('sum')
#data.loc[data['past_readmits_gt30'] > 0, 'past_readmits_gt30'] -= 1
data['past_readmits_none'] = data.groupby('patient_nbr')['readmit_none'].transform('sum')
#data.loc[data['past_readmits_none'] > 0, 'past_readmits_none'] -= 1

dropped_columns.extend(['readmit_lt30', 'readmit_gt30', 'readmit_none'])
#data['past_readmits_lt30'].head(50)

In [172]:
dropped_columns

['patient_nbr',
 'encounter_id',
 'gender',
 'diag_1',
 'diag_2',
 'diag_3',
 'diag_1_n',
 'diag_2_n',
 'diag_3_n',
 'change',
 'readmit_lt30',
 'readmit_gt30',
 'readmit_none']

In [173]:
data = data.drop(columns=dropped_columns)
data.to_csv("./data/data_processed.csv" , index=False)